What now???

* Implement macro-MAE
* Tie-in to NNI API for parameter search-time!
* Compare CORAL to vanilla-OrdReg to classfication
* Retrain the best model(s) with cross-validation

* Similarly, architecture search for CNNs. (Likely CORAL ?)
* Experiment with LSTM funkiness. Consider ELMO and friends.
* Experiment with "language modeling" with problem sentences; generate problems and classify with the above.
* Quantifying uncertainty?

Side quests.

* Dealing with unbalanced data. Using other attributes such as rating.
* Data augmentation!
* Name generation

Far into the future.

* Deep generative models: VAEs, GANs.
* Fun website or interface to bring this all together.

Research

* http://cs229.stanford.edu/proj2017/final-reports/5232206.pdf

## Preliminaries

In [1]:
import moon.data
import moon.problem

In [2]:
probs = moon.data.read_problems('data/cleaned_probs.csv')
import numpy as np

In [9]:
x = np.array([p.array for p in probs])
y = np.array([p.grade.ordinal_rank for p in probs])

print(x.shape, y.shape)

split = 0.2
indices = np.arange(len(probs))
np.random.shuffle(indices)
n_test = int(len(probs) * split)

x_train, y_train = x[indices[n_test:]], y[indices[n_test:]]
x_test, y_test = x[indices[0:n_test]], y[indices[0:n_test]]
print((x_train.shape, y_train.shape), (x_test.shape, y_test.shape))

(30991, 18, 11) (30991, 16)
((24793, 18, 11), (24793, 16)) ((6198, 18, 11), (6198, 16))


In [4]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.backend as K

def accuracy_k_coral(k):
    def fn(y_true, y_pred):
        pred_rank = tf.reduce_sum(tf.cast(y_pred > 0.5, 'float32'), axis=-1)
        true_rank = tf.reduce_sum(y_true, axis=-1)
        score_bools = tf.abs(pred_rank - true_rank) <= k
        return tf.reduce_mean(tf.cast(score_bools, 'float32'), axis=-1)
    fn.__name__ = 'acc' if k == 0 else f'CS{k}'  # https://stackoverflow.com/questions/57910680/how-to-name-custom-metrics-in-keras-fit-output
    return fn

def mae_coral():
    def fn(y_true, y_pred):
        pred_rank = tf.reduce_sum(tf.cast(y_pred > 0.5, 'float32'), axis=-1)
        true_rank = tf.reduce_sum(y_true, axis=-1)
        dist_abs = tf.abs(pred_rank - true_rank)
        return tf.reduce_mean(dist_abs, axis=-1)
    fn.__name__ = 'mae'
    return fn

def rmse_coral():
    def fn(y_true, y_pred):
        pred_rank = tf.reduce_sum(tf.cast(y_pred > 0.5, 'float32'), axis=-1)
        true_rank = tf.reduce_sum(y_true, axis=-1)
        dist_sqr = (pred_rank - true_rank)**2
        return tf.math.sqrt(tf.reduce_mean(dist_sqr, axis=-1))
    fn.__name__ = 'rmse'
    return fn

# see
# https://arxiv.org/abs/1901.07884 and https://github.com/Raschka-research-group/coral-cnn/issues/9
# more background https://arxiv.org/abs/1705.05278

class CoralOutput(tf.keras.layers.Layer):
  def __init__(self, output_len):
    super(CoralOutput, self).__init__()
    self.output_len = output_len

  def build(self, input_shape):
    self.kernel = self.add_weight(
        'kernel',
        shape=[int(input_shape[-1]), 1],
        initializer='glorot_uniform',
        dtype='float32',
        trainable=True
    )
    self.biases = self.add_weight(
        'biases',
        shape=[self.output_len,],
        initializer='zeros',
        dtype='float32',
        trainable=True
    )

  def call(self, input):
    fc = tf.matmul(input, self.kernel)
    fc = tf.tile(fc, [1, self.output_len])
    logits = tf.nn.bias_add(fc, self.biases, name='logits')
    probits = tf.math.sigmoid(logits, name='probits')
    return logits, probits


def _task_importance_weighting(y):
    n_ranks = y.shape[-1]
    ranks = 1 + np.sum(y, axis=-1)
    n_examples = len(y)
    m = np.zeros(n_ranks)
    for k in range(n_ranks):
        s_k = np.sum(ranks > (k + 1))
        m[k] = np.sqrt(max(s_k, n_examples - s_k))
    return (m / np.max(m))


# # adapation of https://github.com/Raschka-research-group/coral-cnn/blob/master/model-code/resnet34/cacd-coral.py#L326
# the authors describe this as "weighted cross-entropy" of the K-1 binary classifiers.
# note (1) we calculate on the output logits, (2) the form in code differs from the form
# in the paper for numerical stability.
def coral_loss(rank_data=None):
    importance = 1 if rank_data is None else _task_importance_weighting(rank_data)
    def loss_logits(y_true, y_pred):
        unweighted = (tf.math.log_sigmoid(y_pred) * y_true) + (tf.math.log_sigmoid(y_pred) - y_pred) * (1 - y_true)
        return tf.reduce_mean(-1 * tf.reduce_sum(importance * unweighted, axis=1))
    return loss_logits

## Initial toying-around with training

In [ ]:
p = .5
input_shape = moon.problem.Problem.GRID_SHAPE
hiddens = [20]
hidden_activation = 'swish'
output_len = y.shape[-1]  # moon.problem.Grade.N_GRADES - 1
output_activation = 'sigmoid'

metrics = {'probits': [accuracy_k_coral(0), accuracy_k_coral(1), accuracy_k_coral(2), mae_coral(), rmse_coral()]}
loss = {'logits': coral_loss(rank_data=y)}
adam_lr = 1e-3
optim = tf.keras.optimizers.Adam(lr=adam_lr)

in_x = layers.Input(shape=input_shape)
features = layers.Flatten()(in_x)
for nodes in hiddens:
    features = layers.Dense(nodes)(features)
    features = layers.Activation(hidden_activation)(features)
    if p > 0: features = layers.Dropout(p)(features)
out = CoralOutput(output_len)(features)
logits, probits = layers.Lambda(lambda x: x, name='logits')(out[0]), layers.Lambda(lambda x: x, name='probits')(out[1])

model = tf.keras.Model(in_x, [logits, probits], name='mlp_ord_regress')
model.compile(loss=loss, optimizer=optim, metrics=metrics)
# model.summary()

callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, verbose=1)]
batch_size = 64
max_epochs = 300

history = model.fit(x_train, y_train, batch_size=batch_size, validation_data=(x_test, y_test), epochs=max_epochs, callbacks=callbacks)

# [64. 64], swish, 1e-4, weighted : e222
# loss: 1.9850 - logits_loss: 1.9850 - probits_acc: 0.3914 - probits_CS1: 0.7283 - probits_CS2: 0.9021 - probits_mae: 1.0237 - probits_rmse: 1.4996 - val_loss: 1.9964 - val_logits_loss: 1.9964 - val_probits_acc: 0.4009 - val_probits_CS1: 0.7228 - val_probits_CS2: 0.8974 - val_probits_mae: 1.0311 - val_probits_rmse: 1.5261

# [32, 32], swish, 1e-4, unweighted
# loss: 2.3904 - logits_loss: 2.3904 - probits_acc: 0.3860 - probits_CS1: 0.7195 - probits_CS2: 0.8967 - val_loss: 2.3429 - val_logits_loss: 2.3429 - val_probits_acc: 0.4045 - val_probits_CS1: 0.7236 - val_probits_CS2: 0.8988

# [32, 32], swish, 1e-3, weighted : e69
# loss: 1.9615 - logits_loss: 1.9615 - probits_acc: 0.3893 - probits_CS1: 0.7298 - probits_CS2: 0.9044 - val_loss: 1.9623 - val_logits_loss: 1.9623 - val_probits_acc: 0.4011 - val_probits_CS1: 0.7304 - val_probits_CS2: 0.9017

# [32, 32], swish, 1e-3, unweighted : e59
# loss: 2.3329 - logits_loss: 2.3329 - probits_acc: 0.3941 - probits_CS1: 0.7271 - probits_CS2: 0.9024 - val_loss: 2.3219 - val_logits_loss: 2.3219 - val_probits_acc: 0.3998 - val_probits_CS1: 0.7296 - val_probits_CS2: 0.9029

# [32, 32], relu, 1e-3, weighted : e39
# loss: 1.9442 - logits_loss: 1.9442 - probits_acc: 0.3801 - probits_CS1: 0.7346 - probits_CS2: 0.9086 - val_loss: 2.0306 - val_logits_loss: 2.0306 - val_probits_acc: 0.3871 - val_probits_CS1: 0.7162 - val_probits_CS2: 0.8927

# [20], swish, 1e-3, weighted, p=.5 : e56
# loss: 2.0464 - logits_loss: 2.0464 - probits_acc: 0.3785 - probits_CS1: 0.7118 - probits_CS2: 0.8934 - probits_mae: 1.0683 - probits_rmse: 1.5502 - val_loss: 2.0134 - val_logits_loss: 2.0134 - val_probits_acc: 0.3918 - val_probits_CS1: 0.7149 - val_probits_CS2: 0.8949 - val_probits_mae: 1.0516 - val_probits_rmse: 1.5428

In [ ]:
print(history.history.keys())  # stats
print(len(history.history['loss']))  # epochs

In [ ]:
y_true = y_test[[2, 4, 8, 16, 32, 64, 128]]
y_pred = model.predict(x_test[[2, 4, 8, 16, 32, 64, 128]])

print(1+np.sum(y_true, axis=-1))
print(1+np.sum(y_pred[1] > .5, axis=-1))
print(y_pred[1])

In [ ]:
print(_task_importance_weighting(y))

## Random Parameter Search + CV

In [6]:
def build_model_coral(input_shape, hiddens, hidden_activation, p, output_activation, output_len, adam_lr, loss):
    in_x = layers.Input(shape=input_shape)
    features = layers.Flatten()(in_x)
    for nodes in hiddens:
        features = layers.Dense(nodes)(features)
        features = layers.Activation(hidden_activation)(features)
        if p > 0: features = layers.Dropout(p)(features)
    out = CoralOutput(output_len)(features)
    logits, probits = layers.Lambda(lambda x: x, name='logits')(out[0]), layers.Lambda(lambda x: x, name='probits')(out[1])
    model = tf.keras.Model(in_x, [logits, probits], name='mlp_consistentranklogits')
    
    metrics = {'probits': [accuracy_k_coral(0), accuracy_k_coral(1), accuracy_k_coral(2), mae_coral(), rmse_coral()]}
    model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(lr=adam_lr), metrics=metrics)
    return model

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

def my_StratifiedKFold(x, y, folds=5, seed=0):
    label_encoder = LabelEncoder()
    ylabels = label_encoder.fit_transform(y)
    cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=0)
    splits = cv.split(x, ylabels)
    for i in range(folds):
        yield next(splits)

In [7]:
from sklearn.model_selection import ParameterSampler
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

np.random.seed(21)

params = dict(
    input_shape = [moon.problem.Problem.GRID_SHAPE],
    hiddens = [[16], [32], [16, 16], [32, 32], [64, 64], [32, 32, 32], [64, 64, 64]],
    hidden_activation = ['swish'],
    p = [.4],
    output_activation = ['sigmoid'],
    output_len = [y.shape[-1]],  # moon.problem.Grade.N_GRADES - 1
    adam_lr = [1e-3],
    batch_size = [32, 64, 96],
    epochs = [30, 50, 75],
    rank_data = [True]
)
n_searches = 100
results = []

for p in ParameterSampler(params, n_iter=n_searches):
    print(f'\n### SEARCH: PARAMS {p}')
    batch_size = p.pop('batch_size')
    epochs = p.pop('epochs')
    rank_data = p.pop('rank_data')
    cv_splitter = my_StratifiedKFold(x, [p.grade.grade for p in probs], folds=5)
    for i, (idx_train, idx_test) in enumerate(cv_splitter):
        print(f'## CV: FOLD {i}... ')
        
        loss = {'logits': coral_loss(rank_data=y[idx_train] if rank_data else None)}    
        model = build_model_coral(**p, loss=loss)
        hist = model.fit(x[idx_train], y[idx_train], batch_size=batch_size, validation_data=(x[idx_test], y[idx_test]), epochs=epochs, verbose=0)
        
        metrics_final_epoch = {x: hist.history[x][-1] for x in hist.history}
        p_rebuilt = {**p, 'batch_size': batch_size, 'epochs': epochs, 'rank_data': rank_data}
        result = {'fold': i, **p_rebuilt, **metrics_final_epoch}
        results.append(result)
        
        print(' '.join([f'{x}:{metrics_final_epoch[x]:.3f}' for x in metrics_final_epoch]))

/home/cove/Documents/Projects/moon/venv/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 63 is smaller than n_iter=100. Running 63 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/home/cove/Documents/Projects/moon/venv/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"



### SEARCH: PARAMS {'rank_data': True, 'p': 0.4, 'output_len': 16, 'output_activation': 'sigmoid', 'input_shape': (18, 11), 'hiddens': [16], 'hidden_activation': 'swish', 'epochs': 30, 'batch_size': 32, 'adam_lr': 0.001}
## CV: FOLD 0... 
loss:2.042 logits_loss:2.042 probits_acc:0.379 probits_CS1:0.716 probits_CS2:0.894 probits_mae:1.065 probits_rmse:1.541 val_loss:1.971 val_logits_loss:1.971 val_probits_acc:0.392 val_probits_CS1:0.729 val_probits_CS2:0.904 val_probits_mae:1.020 val_probits_rmse:1.476
## CV: FOLD 1... 
loss:2.045 logits_loss:2.045 probits_acc:0.379 probits_CS1:0.716 probits_CS2:0.894 probits_mae:1.064 probits_rmse:1.536 val_loss:1.974 val_logits_loss:1.974 val_probits_acc:0.396 val_probits_CS1:0.731 val_probits_CS2:0.904 val_probits_mae:1.015 val_probits_rmse:1.474
## CV: FOLD 2... 
loss:2.045 logits_loss:2.045 probits_acc:0.380 probits_CS1:0.716 probits_CS2:0.893 probits_mae:1.064 probits_rmse:1.536 val_loss:1.991 val_logits_loss:1.991 val_probits_acc:0.392 val_probi

/home/cove/Documents/Projects/moon/venv/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


loss:1.967 logits_loss:1.967 probits_acc:0.393 probits_CS1:0.729 probits_CS2:0.903 probits_mae:1.020 probits_rmse:1.485 val_loss:1.938 val_logits_loss:1.938 val_probits_acc:0.402 val_probits_CS1:0.733 val_probits_CS2:0.908 val_probits_mae:1.003 val_probits_rmse:1.462
## CV: FOLD 1... 
loss:1.977 logits_loss:1.977 probits_acc:0.390 probits_CS1:0.725 probits_CS2:0.900 probits_mae:1.031 probits_rmse:1.498 val_loss:1.944 val_logits_loss:1.944 val_probits_acc:0.404 val_probits_CS1:0.735 val_probits_CS2:0.904 val_probits_mae:1.003 val_probits_rmse:1.468
## CV: FOLD 2... 
loss:1.962 logits_loss:1.962 probits_acc:0.392 probits_CS1:0.726 probits_CS2:0.902 probits_mae:1.023 probits_rmse:1.486 val_loss:1.958 val_logits_loss:1.958 val_probits_acc:0.395 val_probits_CS1:0.736 val_probits_CS2:0.903 val_probits_mae:1.010 val_probits_rmse:1.468
## CV: FOLD 3... 
loss:1.975 logits_loss:1.975 probits_acc:0.392 probits_CS1:0.724 probits_CS2:0.902 probits_mae:1.029 probits_rmse:1.493 val_loss:1.941 val_log

/home/cove/Documents/Projects/moon/venv/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


...

loss:1.866 logits_loss:1.866 probits_acc:0.402 probits_CS1:0.743 probits_CS2:0.917 probits_mae:0.973 probits_rmse:1.431 val_loss:1.908 val_logits_loss:1.908 val_probits_acc:0.407 val_probits_CS1:0.743 val_probits_CS2:0.909 val_probits_mae:0.984 val_probits_rmse:1.459
## CV: FOLD 1... 
loss:1.858 logits_loss:1.858 probits_acc:0.407 probits_CS1:0.742 probits_CS2:0.917 probits_mae:0.968 probits_rmse:1.425 val_loss:1.905 val_logits_loss:1.905 val_probits_acc:0.408 val_probits_CS1:0.739 val_probits_CS2:0.912 val_probits_mae:0.981 val_probits_rmse:1.450
## CV: FOLD 2... 
loss:1.848 logits_loss:1.848 probits_acc:0.407 probits_CS1:0.747 probits_CS2:0.918 probits_mae:0.962 probits_rmse:1.419 val_loss:1.920 val_logits_loss:1.920 val_probits_acc:0.400 val_probits_CS1:0.744 val_probits_CS2:0.911 val_probits_mae:0.986 val_probits_rmse:1.458
## CV: FOLD 3... 
loss:1.861 logits_loss:1.861 probits_acc:0.405 probits_CS1:0.744 probits_CS2:0.915 probits_mae:0.969 probits_rmse:1.425 val_loss:1.894 val_log

/home/cove/Documents/Projects/moon/venv/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


loss:1.988 logits_loss:1.988 probits_acc:0.387 probits_CS1:0.724 probits_CS2:0.900 probits_mae:1.036 probits_rmse:1.515 val_loss:1.929 val_logits_loss:1.929 val_probits_acc:0.405 val_probits_CS1:0.733 val_probits_CS2:0.908 val_probits_mae:1.000 val_probits_rmse:1.481
## CV: FOLD 1... 
loss:1.995 logits_loss:1.995 probits_acc:0.382 probits_CS1:0.723 probits_CS2:0.903 probits_mae:1.040 probits_rmse:1.515 val_loss:1.943 val_logits_loss:1.943 val_probits_acc:0.412 val_probits_CS1:0.726 val_probits_CS2:0.904 val_probits_mae:1.005 val_probits_rmse:1.492
## CV: FOLD 2... 
loss:1.994 logits_loss:1.994 probits_acc:0.388 probits_CS1:0.721 probits_CS2:0.900 probits_mae:1.039 probits_rmse:1.522 val_loss:1.950 val_logits_loss:1.950 val_probits_acc:0.400 val_probits_CS1:0.732 val_probits_CS2:0.900 val_probits_mae:1.013 val_probits_rmse:1.498
## CV: FOLD 3... 
loss:1.992 logits_loss:1.992 probits_acc:0.388 probits_CS1:0.724 probits_CS2:0.900 probits_mae:1.035 probits_rmse:1.516 val_loss:1.922 val_log

/home/cove/Documents/Projects/moon/venv/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


loss:1.894 logits_loss:1.894 probits_acc:0.397 probits_CS1:0.737 probits_CS2:0.914 probits_mae:0.989 probits_rmse:1.450 val_loss:1.913 val_logits_loss:1.913 val_probits_acc:0.399 val_probits_CS1:0.738 val_probits_CS2:0.909 val_probits_mae:0.996 val_probits_rmse:1.463
## CV: FOLD 1... 
loss:1.870 logits_loss:1.870 probits_acc:0.405 probits_CS1:0.747 probits_CS2:0.916 probits_mae:0.969 probits_rmse:1.430 val_loss:1.913 val_logits_loss:1.913 val_probits_acc:0.409 val_probits_CS1:0.738 val_probits_CS2:0.912 val_probits_mae:0.984 val_probits_rmse:1.458
## CV: FOLD 2... 
loss:1.878 logits_loss:1.878 probits_acc:0.399 probits_CS1:0.740 probits_CS2:0.913 probits_mae:0.983 probits_rmse:1.441 val_loss:1.923 val_logits_loss:1.923 val_probits_acc:0.404 val_probits_CS1:0.740 val_probits_CS2:0.909 val_probits_mae:0.990 val_probits_rmse:1.468
## CV: FOLD 3... 
loss:1.902 logits_loss:1.902 probits_acc:0.398 probits_CS1:0.740 probits_CS2:0.913 probits_mae:0.988 probits_rmse:1.451 val_loss:1.897 val_log

In [10]:
import pandas as pd

df = pd.DataFrame.from_records(results)
df.to_csv('mlp_coral_flattened_weighted_1e.3.csv', index=False)

Other playing around...

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# split is [train, validate, test]
def split_data(probs, split=[.6, .2, .2], seed=0):
    # Stratify-split the data first into train and other
    ranks = [p.grade.rank for p in probs]
    probs_train, probs_other = train_test_split(probs, train_size=split[0], stratify=ranks, shuffle=True, random_state=seed)
    # Then into calidation and test
    ranks_other = [p.grade.rank for p in probs_other]
    probs_val, probs_test = train_test_split(probs_other, train_size=split[1]/(split[1]+split[2]), stratify=ranks_other, shuffle=True, random_state=seed)
    return probs_train, probs_val, probs_test

Does this splitter work alright?

In [5]:
s = split_data(probs)
print(len(s[0]), len(s[1]), len(s[2]))  # sum is 30991
print(len(probs))  # Is 30991

# for g in moon.problem.Grade.ALL_GRADES:
#     print(np.mean([p.grade.grade == g for p in s[0]]))
#     print(np.mean([p.grade.grade == g for p in s[1]]))
#     print(np.mean([p.grade.grade == g for p in s[2]]))
    
names0, names1, names2 = set([p.name for p in s[0]]), set([p.name for p in s[1]]), set([p.name for p in s[2]])
print(names0 & names1)
print(names0 & names2)
print(names1 & names2)

18594 6198 6199
30991
{'ATAUALPA', 'SURESHOT'}
{'DISCOVERY', '?'}
{'VARIANT', 'ПЕРЕД ШАШЛЫЧКОМ!', 'FRIENDS'}


Yes---a manual look into things shows that these problem names are repeated in the dataset. (Interestingly, it mostly looks like separate users who decide to clone a problem and give it a new grade, or otherwise have it for themselves.)

In [3]:
import os
import numpy as np
import tensorflow as tf
import random as random

import moon.learning.mlp
import moon.learning.coral
import moon.learning.util
import moon.problem
from importlib import reload
reload(moon.learning.mlp)
reload(moon.learning.coral)
reload(moon.learning.util)
reload(moon.data)

params = dict(
    # Trainer params
    seed = 21,
    batch_size = 64,
    max_epochs = 5,
    # Model params
    flattened_input = True,
    hidden_dim = 16,
    hidden_layers = 1,
    hidden_activation = 'swish',
    dropout_p = 0.5,
    adam_lr = 1e-3,
    output_type = 'categorical_xentropy',
#     output_type = 'ordinal_vanilla',
#     output_type = 'ordinal_coral',
    weighted_loss = True
)

os.environ['PYTHONHASHSEED'] = str(params['seed'])
random.seed(params['seed'])
np.random.seed(params['seed'])
tf.random.set_seed(params['seed'])

splits = moon.data.split_data(probs, seed=params['seed'])
x_train, x_val, x_test = [np.array([p.array for p in probs]) for probs in splits]
# y_train, y_val, y_test = [np.array([p.grade.ordinal_rank for p in probs]) for probs in splits]
y_train, y_val, y_test = [np.array([p.grade.categorical for p in probs]) for probs in splits]

input_shape = moon.problem.Problem.GRID_SHAPE if params['flattened_input'] else (*moon.problem.Problem.GRID_SHAPE, 3)
# output_len = moon.problem.Grade.N_GRADES - 1  # y.shape[-1]
output_len = moon.problem.Grade.N_GRADES  # y.shape[-1]
y_train_for_model = y_train if params['weighted_loss'] else None

m = moon.learning.mlp.build_keras_mlp(
    input_shape=input_shape,
    hidden_dim=params['hidden_dim'],
    hidden_layers=params['hidden_layers'],
    hidden_activation=params['hidden_activation'],
    dropout_p=params['dropout_p'],
    adam_lr=params['adam_lr'],
    output_type=params['output_type'],
    output_len=output_len,
    y_train=y_train_for_model)

In [4]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, verbose=1)]
batch_size = params['batch_size']
max_epochs = params['max_epochs']

history = m.fit(x_train, y_train, batch_size=batch_size, validation_data=(x_val, y_val), epochs=max_epochs, callbacks=callbacks)

Epoch 1/5
291/291 [==============================] - 1s 2ms/step - loss: 2.2760 - probits_loss: 2.2760 - probits_mae: 1.6254 - probits_acc1: 0.4521 - probits_acc: 0.2709 - val_loss: 1.8439 - val_probits_loss: 1.8439 - val_probits_mae: 1.3814 - val_probits_acc1: 0.4969 - val_probits_acc: 0.3401
Epoch 2/5
291/291 [==============================] - 1s 2ms/step - loss: 1.8646 - probits_loss: 1.8646 - probits_mae: 1.1890 - probits_acc1: 0.5258 - probits_acc: 0.3433 - val_loss: 1.7315 - val_probits_loss: 1.7315 - val_probits_mae: 1.0206 - val_probits_acc1: 0.5540 - val_probits_acc: 0.3610
Epoch 3/5
291/291 [==============================] - 1s 2ms/step - loss: 1.7797 - probits_loss: 1.7797 - probits_mae: 1.0241 - probits_acc1: 0.5616 - probits_acc: 0.3597 - val_loss: 1.6882 - val_probits_loss: 1.6882 - val_probits_mae: 0.9897 - val_probits_acc1: 0.5883 - val_probits_acc: 0.3765
Epoch 4/5
291/291 [==============================] - 0s 1ms/step - loss: 1.7406 - probits_loss: 1.7406 - probits_ma

In [6]:
m.evaluate(x_test, y_test, return_dict=True)

194/194 [==============================] - 0s 879us/step - loss: 1.6473 - probits_loss: 1.6473 - probits_mae: 0.9330 - probits_acc1: 0.6105 - probits_acc: 0.3869


{'loss': 1.6473280191421509,
 'probits_loss': 1.6473280191421509,
 'probits_mae': 0.9329897165298462,
 'probits_acc1': 0.6105446219444275,
 'probits_acc': 0.3869411051273346}